In [1]:
cd Desktop/Summer\ research/fem/

/Users/wuyifan/Desktop/Summer research/fem


In [3]:
import numpy as np
import math
import igl
import meshplot as mp
import wildmeshing as wm
import polyfempy as pf

V, F = igl.read_triangle_mesh("data/bunny_s.obj")
#p1 = mp.plot(V, F, shading={"wireframe": True}, return_plot=True)
#p1.add_points(V, shading={"point_color": "blue", "point_size": 0.5});
#p1.save("cylinder")

#wm.tetrahedralize("data/bunny_s.obj", "out_bunny_s.mesh", mute_log=True)
solver = pf.Solver()
solver.load_mesh_from_path("out_bunny_s.mesh")

[2020-05-26 16:00:37.497] [polyfem] [info] Loading mesh...
[2020-05-26 16:00:37.507] [geogram] [info] Loading file out_bunny_s.mesh...
[2020-05-26 16:00:37.621] [geogram] [info] (FP64) nb_v:4896 nb_e:0 nb_f:6068 nb_b:0 tri:1 dim:3
[2020-05-26 16:00:37.621] [geogram] [info]  nb_tets:18832
[2020-05-26 16:00:37.621] [geogram] [info] Attributes on vertices: point[3]
[2020-05-26 16:00:37.948] [polyfem] [info] mesh bb min [-0.0943395, 0.0319048, -0.0616427], max [0.0607677, 0.186996, 0.0587146]
[2020-05-26 16:00:37.949] [polyfem] [info]  took 0.452898s


In [19]:
minn = np.min(V, axis=0)
maxx = np.max(V, axis=0)

def sideset(p):
    if p[1]<=minn[1]+0.005:
        return 1
    elif p[1]>=maxx[1]-0.03:
        return 2
    return 3
solver.set_boundary_side_set_from_bary(sideset)
ps, ts, s = solver.get_boundary_sidesets()

e1 = ps[s[:,0] == 1, :]
e2 = ps[s[:,0] == 2, :]
e3 = ps[s[:,0] == 3, :]

col = np.zeros_like(s); col[s==2] = 2; col[s==3] = 3
plot2 = mp.plot(ps, ts, col, return_plot=True)
#plot2.add_points(e2, shading={"point_color": "red", "point_size": 0.05})
#plot2.add_points(e3, shading={"point_color": "blue", "point_size": 0.05})
plot2.save("col_bunny_s")

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016785…

Plot saved to file col_bunny_s.html.


In [20]:
settings = pf.Settings()
problem = pf.Problem()

settings.set_pde(pf.PDEs.LinearElasticity)

settings.set_material_params("E", 200)
settings.set_material_params("nu", 0.35)

# set the displacement value for the sideset id
problem.set_displacement(1, [0, 0, 0])
problem.set_force(3, [2, 0, 0])

settings.set_problem(problem)

solver.settings(settings)
solver.solve()

p, tri, disp = solver.get_sampled_solution()
#print(len(p),len(ps),len(s))
m = np.linalg.norm(disp, axis=1)

[2020-05-26 16:07:20.347] [polyfem] [info] simplex_count: 	18832
[2020-05-26 16:07:20.347] [polyfem] [info] regular_count: 	0
[2020-05-26 16:07:20.347] [polyfem] [info] regular_boundary_count: 	0
[2020-05-26 16:07:20.347] [polyfem] [info] simple_singular_count: 	0
[2020-05-26 16:07:20.347] [polyfem] [info] multi_singular_count: 	0
[2020-05-26 16:07:20.347] [polyfem] [info] boundary_count: 	0
[2020-05-26 16:07:20.347] [polyfem] [info] multi_singular_boundary_count: 	0
[2020-05-26 16:07:20.347] [polyfem] [info] non_regular_count: 	0
[2020-05-26 16:07:20.347] [polyfem] [info] non_regular_boundary_count: 	0
[2020-05-26 16:07:20.347] [polyfem] [info] undefined_count: 	0
[2020-05-26 16:07:20.347] [polyfem] [info] total count:	 18832
[2020-05-26 16:07:20.347] [polyfem] [info] Building isoparametric basis...
[2020-05-26 16:07:20.428] [polyfem] [info] Computing polygonal basis...
[2020-05-26 16:07:20.428] [polyfem] [info]  took 1.5812e-05s
[2020-05-26 16:07:20.433] [polyfem] [info] hmin: 0.0004

In [21]:
plot = mp.plot(v=p+disp, f=tri, c=m, return_plot=True)
plot.save("bunny_")

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.001916…

Plot saved to file bunny_[1.5,0,0].html.
